# Requires Tensorflow 1.15!

In [1]:
import os
from pathlib import Path
import requests
import tarfile
from tqdm import tqdm
import json
import numpy as np
import shutil
import pandas as pd
import librosa
import librosa.display
import sounddevice as sd
sd.default.samplerate = 16000
import torch
import torch.utils.data as td
from mlp64 import data
from mlp64 import st
import torchaudio as ta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import Audio

import scipy.fft as spfft
librosa.set_fftlib(spfft)

pd.set_option('display.max_columns', 30)

In [2]:
sr = 16000
n_fft = 800
cqt = lambda y: np.abs(librosa.cqt(y, sr=16000, hop_length=192))
ms = ta.transforms.MelSpectrogram(n_fft=800, hop_length=160)
s = lambda y: torch.as_tensor(np.log1p(np.abs(librosa.stft(y, n_fft))))
transform = cqt

In [3]:
path = Path("/home/tim/Desktop/MLP64/dataset/nsynth-test/")
vdf = data.create_dataset_df(path / "examples.json")
files = data.dftostr(vdf)

JSON loaded into DataFrame!


In [4]:
scontent = 'keyboard_acoustic_004-079-127.wav'
sstyle = 'brass_acoustic_046-059-075.wav'
others = ['bass_electronic_018-024-025.wav', 'flute_acoustic_002-107-100.wav']

In [5]:
npdata = data.load_files(path / "audio", [scontent, sstyle] + others)

In [61]:
# Copy files into baselines folder
shutil.copy(path / "audio" / scontent, "../baselines/source0_" + scontent)
shutil.copy(path / "audio" / sstyle, "../baselines/target0_" + sstyle)
shutil.copy(path / "audio" / others[0], "../baselines/target1_" + others[0])
shutil.copy(path / "audio" / others[1], "../baselines/target2_" + others[1])

'../baselines/target2_flute_acoustic_002-107-100.wav'

In [6]:
sd.play(npdata[:, 1])

In [8]:
from audio_style_transfer.models import timedomain

In [60]:
timedomain.run?

Signature:
timedomain.run(
    content_fname,
    style_fname,
    output_fname,
    norm=False,
    input_features=['real', 'imag', 'mags'],
    n_fft=4096,
    n_layers=1,
    n_filters=4096,
    hop_length=256,
    alpha=0.05,
    k_w=15,
    k_h=3,
    optimizer='bfgs',
    stride=1,
    iterations=300,
    sr=22050,
)
Docstring: <no docstring>
File:      ~/miniconda3/envs/mlp/lib/python3.7/site-packages/audio_style_transfer/models/timedomain.py
Type:      function


In [17]:
out_filename = "../baselines/pkmital/source0_target1_def.wav"

In [18]:
timedomain.run("../baselines/source0_keyboard_acoustic_004-079-127.wav", "../baselines/target1_bass_electronic_018-024-025.wav", out_filename, sr=16000)

Started optimization.
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 5438296.000000
  Number of iterations: 300
  Number of functions evaluations: 308


In [19]:
sd.play(librosa.load(out_filename, sr=16000)[0])

In [7]:
from audio_style_transfer.models import uylanov

In [8]:
out_filename = "../baselines/ulyanov/source0_target1_calcfft.wav"
uylanov.run("../baselines/source0_keyboard_acoustic_004-079-127.wav", "../baselines/target1_bass_electronic_018-024-025.wav", out_filename, n_fft=800, hop_length=190, sr=16000)

AttributeError: module 'tensorflow' has no attribute 'Session'

In [24]:
sd.play(librosa.load(out_filename, sr=16000)[0])